In [1]:
import pytesseract
import cv2
from PIL import Image 
import re
import pandas as pd
import fitz
import os
import boto3
import io
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import numpy as np

F:\anaconda3\envs\python_3_7\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')



# Code to Convert PDF to PNG

In [2]:
def convert_pdf2img(input_file: str,pages=None):
    """Converts pdf to image and generates a file by page"""
    # Open the document
    pdfIn = fitz.open(input_file)
    output_files = []
    # Iterate throughout the pages
    for pg in range(pdfIn.pageCount):
        if str(pages) != str(None):
            if str(pg) not in str(pages):
                continue
        # Select a page
        page = pdfIn[pg]
        rotate = int(0)
        # PDF Page is converted into a whole picture 1056*816 and then for each picture a screenshot is taken.
        # zoom = 1.33333333 -----> Image size = 1056*816
        # zoom = 2 ---> 2 * Default Resolution (text is clear, image text is hard to read)    = filesize small / Image size = 1584*1224
        # zoom = 4 ---> 4 * Default Resolution (text is clear, image text is barely readable) = filesize large
        # zoom = 8 ---> 8 * Default Resolution (text is clear, image text is readable) = filesize large
        zoom_x = 2
        zoom_y = 2
        # The zoom factor is equal to 2 in order to make text clear
        # Pre-rotate is to rotate if needed.
        mat = fitz.Matrix(zoom_x, zoom_y).preRotate(rotate)
        pix = page.getPixmap(matrix=mat, alpha=False)
        path = 'Makro Invoices/Makro Invoices/New/'
        output_file = f"{path+os.path.splitext(os.path.basename(input_file))[0]}_page{pg+1}.png"
        pix.writePNG(output_file)
        output_files.append(output_file)
    pdfIn.close()
    summary = {
        "File": input_file, "Pages": str(pages), "Output File(s)": str(output_files)
    }
    # Printing Summary
    print("## Summary ########################################################")
    print("\n".join("{}:{}".format(i, j) for i, j in summary.items()))
    print("###################################################################")
    return output_files

In [442]:
convert_pdf2img('Makro Invoices/Makro Invoices/Convert/60064_B38245866 DEFAULT B0134 TYPE_20220203_6_FacturaCompra.pdf')

Deprecation: 'preRotate' removed from class 'Matrix' after v1.19 - use 'prerotate'.
Deprecation: 'getPixmap' removed from class 'Page' after v1.19 - use 'get_pixmap'.


## Summary ########################################################
File:Makro Invoices/Makro Invoices/Convert/60064_B38245866 DEFAULT B0134 TYPE_20220203_6_FacturaCompra.pdf
Pages:None
Output File(s):['Makro Invoices/Makro Invoices/New/60064_B38245866 DEFAULT B0134 TYPE_20220203_6_FacturaCompra_page1.png']
###################################################################


Deprecation: 'writePNG' removed from class 'Pixmap' after v1.19 - use 'save'.


['Makro Invoices/Makro Invoices/New/60064_B38245866 DEFAULT B0134 TYPE_20220203_6_FacturaCompra_page1.png']

In [3]:
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

In [6]:
formats = pd.read_excel('invoice_formats.xlsx')
final = pd.read_excel('Extracted_Data.xlsx')

In [853]:
Ima = "Makro Invoices/Makro Invoices/New/10.11.20_Semillas Batlle SA_561,47_page1.png"

In [854]:
def find_amounts(text):
    amounts = re.findall(r'\d+\.\d{2}\b', text)
    floats = [float(amount) for amount in amounts]
    unique = list(dict.fromkeys(floats))
    if len(unique)==0:
        return 0
    else:
        return max(unique)

In [855]:
%%time
dat = {
    "Vat Number":'',
       "Total":0,
       "Date":'',
       "Document_Type":"",
      "Invoice_Number":''}
img  = cv2.imread(Ima, cv2.COLOR_RGB2GRAY)
text=(pytesseract.image_to_string(img))
text_list = text.split("\n")
vat = []
date = []
tot = []
doc_type = []
print(type(text_list))
dat['Total']=find_amounts(text.replace(',','.'))
for idx, a in enumerate(text_list):
    if(a.strip() != ""):
        

        a = a.replace(',','.')
        if (re.search("[A-Z]{1}-[0-9]{8}",a)):
            b = re.findall("[A-Z]{1}-[0-9]{8}",a)
            dat["Vat Number"] = b[0]
            vat.append(b[0].replace('-',''))
            print(b)
            
        elif (re.search("[A-Z]{1}[0-9]{8}",a)):
            b = re.findall("[A-Z]{1}[0-9]{8}",a)
            dat["Vat Number"] = b[0]
            vat.append(b[0].replace('-',''))
            print(b)
            
        elif (re.search("[0-9]{8}-[A-Z]{1}",a)):
            b = re.findall("[0-9]{8}-[A-Z]{1}",a)
            dat["Vat Number"] = b[0]
            vat.append(b[0].replace('-',''))
        elif (re.search("[0-9]{8}[A-Z]{1}",a)):
            b = re.findall("[0-9]{8}[A-Z]{1}",a)
            dat["Vat Number"] = b[0]
            vat.append(b[0])
    
        elif (re.search("[A-Z]{1}-[0-9]{8}",a)):
            b = re.findall("[A-Z]{1}-[0-9]{8}",a)
            dat["Vat Number"] = b[0]
            vat.append(b[0].replace('-',''))

        elif (re.search("[A-Z]{1}[0-9]{8}",a)):
            b = re.findall("[A-Z]{1}[0-9]{8}",a)
            dat["Vat Number"] = b[0]
            vat.append(b[0])
            
        elif (re.search("[A-Z]{1}[0-9]{2}/[0-9]{6}",a)):
            b = re.findall("[A-Z]{1}[0-9]{2}/[0-9]{6}",a)
            dat["Vat Number"] = b[0]
            vat.append(b[0].replace('/',''))
            
        elif (re.search("[A-Z]{1}-[0-9]{2}.[0-9]{3}.[0-9]{3}",a)):
            b = re.findall("[A-Z]{1}-[0-9]{2}.[0-9]{3}.[0-9]{3}",a)
            di = b[0].replace('-','')
            dit = di.replace(',','')
            dat["Vat Number"] = dit.replace('.','')
        elif (re.search("[A-Z]{1}-[0-9]{2}/[0-9]{6}",a)):
            b = re.findall("[A-Z]{1}-[0-9]{2}/[0-9]{6}",a)
            di = b[0].replace('-','')
            dit = di.replace(',','')
            dat["Vat Number"] = dit.replace('/','')

        if len(vat)>1:
            dat["Vat Number"] = vat[0]
        else:
            pass
        
        if "ALBARAN" in a or "Albaran" in a:
            dat["Document_Type"] = "Albaran"
            doc_type.append("Albaran")
        elif "Factura" in a or "FACTURA" in a  or "factura" in a :
            dat["Document_Type"] = "Factura"
            doc_type.append("Factura")
        if len(doc_type)>1:
            dat["Document_Type"] = doc_type[0]
        
         #Date section           
        if (re.search("\d{2}/\d{2}/\d{4}",a)):
            b  = re.findall("\d{2}/\d{2}/\d{4}", a)
            dat["Date"] = b[0].replace('/','-')
            date.append(b[0].replace('/','-'))
        elif (re.search("\d{2}-\d{2}-\d{4}",a)):
            b = re.findall("\d{2}-\d{2}-\d{4}",a)
            dat["Date"] = b[0].replace('/','-')
            date.append(b[0].replace('/','-'))
        elif (re.search("\d{2}/\d{2}/\d{2}",a)):
            b = re.findall("\d{2}/\d{2}/\d{2}",a)
            dat["Date"] = b[0].replace('/','-')
            date.append(b[0].replace('/','-'))
        elif (re.search("\d{1}/\d{2}/\d{2}",a)):
            b = re.findall("\d{1}/\d{2}/\d{2}",a)
            dat["Date"] = b[0].replace('/','-')
            date.append(b[0].replace('/','-'))
        
        elif (re.search("[0-9]{2}\.[0-9]{2}\.[0-9]{4}",a)):
            b = re.findall("[0-9]{2}\.[0-9]{2}\.[0-9]{4}",a)
            dat["Date"] = b[0].replace('.','-')
            date.append(b[0].replace('.','-'))
    
        if len(date)>1:
            dat["Date"] = date[0].replace('/','-')

    
    
       
        
        

<class 'list'>
['A25004714']
['B76305333']
['A25004714']
Wall time: 21.3 s


In [856]:
dat

{'Vat Number': 'A25004714',
 'Total': 561.47,
 'Date': '10-11-2020',
 'Document_Type': 'Factura',
 'Invoice_Number': ''}

In [857]:
print(dat['Date'][0:6])

10-11-


In [858]:
dat
if len(dat['Date'])==10:
    pass
elif len(dat['Date'])==8:
    dat['Date'] = dat['Date'][0:6]+'20'+dat['Date'][6:]

In [859]:
dat

{'Vat Number': 'A25004714',
 'Total': 561.47,
 'Date': '10-11-2020',
 'Document_Type': 'Factura',
 'Invoice_Number': ''}

In [860]:
#dat['Date'].replace('/','-')


#dat['Vat Number'] = dat['Vat Number'].replace(',', '')
#dat['Vat Number'] = dat['Vat Number'].replace('.', '')
#dat['Vat Number'] = dat['Vat Number'].replace('-', '') 
#dat['Vat Number'] = dat['Vat Number'].replace('/', '')
#if dat['Total'].isdigit():
#    print('h')

In [49]:
data  =  pd.DataFrame(dat,index=[0])
data['Vat Number']= data['Vat Number'].str.replace(' ', '')
print(data)

  Vat Number  Total        Date Document_Type Invoice_Number
0  B16507295  604.3  24-05-2022                             


In [50]:
data['Total'] =data['Total'].str.replace(',','.')

AttributeError: Can only use .str accessor with string values!

In [51]:
data['Vat Number']= data['Vat Number'].str.replace('-', '')
data['Vat Number']= data['Vat Number'].str.replace(',', '')
data['Vat Number']= data['Vat Number'].str.replace('.', '')
data['Vat Number']= data['Vat Number'].str.replace('/', '')
data

F:\anaconda3\envs\python_3_7\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  This is separate from the ipykernel package so we can avoid doing imports until



,Vat Number,Total,Date,Document_Type,Invoice_Number
0,B16507295,604.3,24-05-2022,,


In [13]:
formats['last']= (formats['nif'].isin(data['Vat Number']))

In [14]:
df2 = formats.query("last == True")

In [15]:
df2

,formato,nombre,nif,last
8892,\d\d\d\d\d\d\d,Isayca Tenerife SLU,B38988242,True
12519,\d\d\d\d\d\d\d,Isayca Tenerife SLU,B38988242,True


In [16]:
inv = 0
for i in df2['formato']:
    inv = i
if inv ==0:
    pass
else:
    factura = re.compile(inv)
    mo = factura.search(text)
    if mo is None:
        pass
    else:
        dat["Invoice_Number"]=mo.group()

In [17]:
dat

{'Vat Number': 'B38988242',
 'Total': '273.22',
 'Date': '3-02-20',
 'Document_Type': 'Factura',
 'Invoice_Number': '1825768'}

In [861]:
im = Image.open(Ima)

In [862]:
buffered = io.BytesIO()
im.save(buffered, format='PNG')
client = boto3.client('textract')
response = client.analyze_document(
    Document={'Bytes': buffered.getvalue()},
    FeatureTypes=['TABLES']
)

In [863]:
def map_blocks(blocks, block_type):
    return {
        block['Id']: block
        for block in blocks
        if block['BlockType'] == block_type
    }

blocks = response['Blocks']
tables = map_blocks(blocks, 'TABLE')
cells = map_blocks(blocks, 'CELL')
words = map_blocks(blocks, 'WORD')
selections = map_blocks(blocks, 'SELECTION_ELEMENT')

def get_children_ids(block):
    for rels in block.get('Relationships', []):
        if rels['Type'] == 'CHILD':
            yield from rels['Ids']
            
dataframes = []

for table in tables.values():

    # Determine all the cells that belong to this table
    table_cells = [cells[cell_id] for cell_id in get_children_ids(table)]

    # Determine the table's number of rows and columns
    n_rows = max(cell['RowIndex'] for cell in table_cells)
    n_cols = max(cell['ColumnIndex'] for cell in table_cells)
    content = [[None for _ in range(n_cols)] for _ in range(n_rows)]

    # Fill in each cell
    for cell in table_cells:
        cell_contents = [
            words[child_id]['Text']
            if child_id in words
            else selections[child_id]['SelectionStatus']
            for child_id in get_children_ids(cell)
        ]
        i = cell['RowIndex'] - 1
        j = cell['ColumnIndex'] - 1
        content[i][j] = ' '.join(cell_contents)

    # We assume that the first row corresponds to the column names
    dataframe = pd.DataFrame(content[1:], columns=content[0])
    dataframes.append(dataframe)

In [867]:
len(dataframes)

2

In [868]:
d = dataframes[0]

In [869]:
d.head(20)

,Alb.,Cantidad Envase,Artículo,I.V.A.,Euro Uni.,Dto.,Euros Línea,Neto Kg/Un Aproximado
0,248011,,,,,,,
1,,"5,00 1Kg Caja",Alfalfa CAPITANA R1,0+0,"6,500","0,00","32,50","6,500"
2,,"10,00 Bols Eco",Calabaza Butternut ECO,0+0,"1,000","30,00","7,00","0,700"
3,,"10,00 Bols Eco",Calabaza Mallorca ECO,0+0,"1,000","30,00","7,00","0,700"
4,,"10,00 Bols Eco",Lechuga Cuatro estaciones - ECO,0+0,"1,000","30,00","7,00","0,700"
5,,"10,00 Bols Eco",Pepino Marketmore 70 - ECO,0+0,"1,000","30,00","7,00","0,700"
6,,"10,00 Bols Eco",Perejil Común 2 (hoja grande) ECO,0+0,"1,000","30,00","7,00","0,700"
7,,"10,00 Bols Eco",Pimiento Largo de Reus - ECO,0+0,"1,000","30,00","7,00","0,700"
8,,"10,00 Bols Eco",Pimiento Dulce Italiano - ECO,0+0,"1,000","30,00","7,00","0,700"
9,,"10,00 Bols Eco",Rabanito medio largo rojo P.BLANCA -,0+0,"1,000","30,00","7,00","0,700"


In [870]:
d.columns

Index(['Alb.', 'Cantidad Envase', 'Artículo', 'I.V.A.', 'Euro Uni.', 'Dto.',
       'Euros Línea', 'Neto Kg/Un Aproximado'],
      dtype='object')

In [871]:
merged_col = False
print(type(d))
for i in (d.columns):
    if i == '':
        print(i)
        merged_col = True

<class 'pandas.core.frame.DataFrame'>


In [872]:
if "Código Descripción" in d.columns:
    print('HI')

In [873]:
def get_desc(desc):
    return desc.split(" ")[0].strip(" ")

In [874]:
if merged_col:
    print("hi")
    if "Código Descripción" in d.columns:
        d['Codigo'] = d["Código Descripción"].apply(lambda x: f"{get_desc(x)}")
        d['Codigo']= d['Codigo'].str.replace('/', '')
        d['Codigo']= d['Codigo'].str.replace(':', '')
        d['Codigo']= d['Codigo'].str.replace('[A-Z]', '')
        d['Codigo']= d['Codigo'].str.replace('[a-z]', '')
        d['DESCRIPCIÓN']= d['Código Descripción'].str.replace('[0-9]', '')
        d['DESCRIPCIÓN']= d['DESCRIPCIÓN']+' '+d.iloc[:,1]
        d =d.drop(d.columns[[0, 1]],axis = 1) 
        first_column = d.pop('Codigo')
        d.insert(0,'Codigo',first_column)
        second_column = d.pop('DESCRIPCIÓN')
        d.insert(1,'DESCRIPCIÓN',second_column)
    else:
        d['Codigo'] = d["Código"].apply(lambda x: f"{get_desc(x)}")
        d['Codigo']= d['Codigo'].str.replace('/', '')
        d['Codigo']= d['Codigo'].str.replace(':', '')
        d['Codigo']= d['Codigo'].str.replace('[A-Z]', '')
        d['Codigo']= d['Codigo'].str.replace('[a-z]', '')
        d['DESCRIPCIÓN']= d['Descripción'].str.replace('[0-9]', '')
        d['DESCRIPCIÓN']= d['DESCRIPCIÓN']+' '+d.iloc[:,1]
        d =d.drop(d.columns[[0, 1]],axis = 1) 
        first_column = d.pop('Codigo')
        d.insert(0,'Codigo',first_column)
        second_column = d.pop('DESCRIPCIÓN')
        d.insert(1,'DESCRIPCIÓN',second_column)
        
        
#    d['Codigo'] = d["Código"].apply(lambda x: f"{get_desc(x)}")
#    d['Codigo']= d['Codigo'].str.replace('/', '')
#    d['Codigo']= d['Codigo'].str.replace('[A-Z]', '')
#    d['Codigo']= d['Codigo'].str.replace('[a-z]', '')
#    d['DESCRIPCIÓN']= d['Código'].str.replace('/', '')
#    d['DESCRIPCIÓN']= d['DESCRIPCIÓN'].str.replace('[0-9]', '')
#    d['DESCRIPCIÓN']= d['DESCRIPCIÓN']+d.iloc[:,1]
#    d =d.drop(d.columns[[0, 1]],axis = 1) 
#    first_column = d.pop('Codigo')
#    d.insert(0,'Codigo',first_column)
#    second_column = d.pop('DESCRIPCIÓN')
#    d.insert(1,'DESCRIPCIÓN',second_column)

In [875]:
if merged_col:
    this_data = d
    print('Yes')

In [878]:
str1 ='DESCRIPCIÓN'
str2 = "Descrip. artículo"
str3 = "Artículo/ Cod.cliente"
str4 = 'DESCRIPCIÓN:'
str5 = "CONCEPTO"
str6 = "UDS"
str7 = "Concepto"
str8 = "Descripción"
str9 = "DESCRIPCION"   
str10 = "Description articulo"
str11 = "Description"
str12 = "DESCRIPCIÓN ARTICULO"
str13 = "Unidades Descripción"
str14 = "PRODUCTO"
str15 = "Articulo"
str16 = 'Denominación /Referencia'
str17 = 'DESCRIPCIÓN'
str18 = 'REF.'
str19 = "Codigo"
str20 = "Lote"
str21 = "MM Num. articulo"
str22 = 'Artículo'
zero = False
one = False
two = False
three = False

if not merged_col:
    if "ARTÍCULO" in dataframes[0].columns or "CODI EAN/CÓDIGO EAN" in dataframes[0].columns or "MM Num. artículo" in dataframes[0].columns  or 'CÓDIGO' in dataframes[0].columns or 'MM Num.' in dataframes[0].columns or str3 in dataframes[0].columns or str4 in dataframes[0].columns or str5 in dataframes[0].columns or str6 in dataframes[0].columns or str7 in dataframes[0].columns or str8 in dataframes[0].columns or str9 in dataframes[0].columns or str10 in dataframes[0].columns or str11 in dataframes[0].columns or str12 in dataframes[0].columns or str13 in dataframes[0].columns or str14 in dataframes[0].columns or str15 in dataframes[0].columns or str16 in dataframes[0].columns or str17 in dataframes[0].columns or str18 in dataframes[0].columns or str19 in dataframes[0].columns or str20 in dataframes[0].columns or str21 in dataframes[0].columns or str22 in dataframes[0].columns:
            this_data = pd.DataFrame(dataframes[0])
            print('---------0')
            zero = True
    if len(dataframes)>1: 
        if "ARTÍCULO" in dataframes[1].columns or "CODI EAN/CÓDIGO EAN" in dataframes[1].columns or "MM Num. artículo" in dataframes[1].columns or 'CÓDIGO' in dataframes[1].columns or 'MM Num.' in dataframes[1].columns or str3 in dataframes[1].columns or str4 in dataframes[1].columns or str5 in dataframes[1].columns or str6 in dataframes[1].columns or str7 in dataframes[1].columns or str8 in dataframes[1].columns or str9 in dataframes[1].columns or str10 in dataframes[1].columns or str11 in dataframes[1].columns or str12 in dataframes[1].columns or str13 in dataframes[1].columns or str14 in dataframes[1].columns or str15 in dataframes[1].columns or str16 in dataframes[1].columns or str17 in dataframes[1].columns or str18 in dataframes[1].columns or str19 in dataframes[1].columns or str20 in dataframes[1].columns or str21 in dataframes[1].columns or str22 in dataframes[1].columns:        
                this_data = pd.DataFrame(dataframes[1])
                print('---------1')
                one = True
    if len(dataframes)>2:        
        if "ARTÍCULO" in dataframes[2].columns or "CODI EAN/CÓDIGO EAN" in dataframes[2].columns or "MM Num. artículo" in dataframes[2].columns or 'CÓDIGO' in dataframes[2].columns or 'MM Num.' in dataframes[2].columns or str3 in dataframes[2].columns or str4 in dataframes[2].columns or str5 in dataframes[0].columns or str6 in dataframes[2].columns or str7 in dataframes[2].columns or str8 in dataframes[2].columns or str9 in dataframes[2].columns or str10 in dataframes[2].columns or str11 in dataframes[2].columns or str12 in dataframes[2].columns or str13 in dataframes[2].columns or str14 in dataframes[2].columns or str15 in dataframes[2].columns or str16 in dataframes[2].columns or str16 in dataframes[2].columns or str18 in dataframes[2].columns or str19 in dataframes[2].columns or str20 in dataframes[2].columns or str21 in dataframes[2].columns or str22 in dataframes[2].columns:
                this_data = pd.DataFrame(dataframes[2])
                print('---------2')
                two = True
    if len(dataframes)>3:
        if "ARTÍCULO" in dataframes[3].columns or "CODI EAN/CÓDIGO EAN" in dataframes[3].columns or "MM Num. artículo" in dataframes[3].columns or 'CÓDIGO' in dataframes[3].columns or 'MM Num.' in dataframes[3].columns or str3 in dataframes[3].columns or str4 in dataframes[3].columns or str5 in dataframes[3].columns or str6 in dataframes[3].columns or str7 in dataframes[3].columns or str8 in dataframes[3].columns or str9 in dataframes[3].columns or str10 in dataframes[3].columns or str11 in dataframes[3].columns or str12 in dataframes[3].columns or str13 in dataframes[3].columns or str14 in dataframes[3].columns or str15 in dataframes[3].columns or str16 in dataframes[3].columns or str16 in dataframes[3].columns or str18 in dataframes[3].columns or str19 in dataframes[3].columns or str20 in dataframes[3].columns or  str21 in dataframes[3].columns or str22 in dataframes[3].columns:
            this_data = pd.DataFrame(dataframes[3])
            print('---------3')
            three = True
else:
    pass
    
this_data


---------0


,Alb.,Cantidad Envase,Artículo,I.V.A.,Euro Uni.,Dto.,Euros Línea,Neto Kg/Un Aproximado
0,248011,,,,,,,
1,,"5,00 1Kg Caja",Alfalfa CAPITANA R1,0+0,"6,500","0,00","32,50","6,500"
2,,"10,00 Bols Eco",Calabaza Butternut ECO,0+0,"1,000","30,00","7,00","0,700"
3,,"10,00 Bols Eco",Calabaza Mallorca ECO,0+0,"1,000","30,00","7,00","0,700"
4,,"10,00 Bols Eco",Lechuga Cuatro estaciones - ECO,0+0,"1,000","30,00","7,00","0,700"
5,,"10,00 Bols Eco",Pepino Marketmore 70 - ECO,0+0,"1,000","30,00","7,00","0,700"
6,,"10,00 Bols Eco",Perejil Común 2 (hoja grande) ECO,0+0,"1,000","30,00","7,00","0,700"
7,,"10,00 Bols Eco",Pimiento Largo de Reus - ECO,0+0,"1,000","30,00","7,00","0,700"
8,,"10,00 Bols Eco",Pimiento Dulce Italiano - ECO,0+0,"1,000","30,00","7,00","0,700"
9,,"10,00 Bols Eco",Rabanito medio largo rojo P.BLANCA -,0+0,"1,000","30,00","7,00","0,700"


In [879]:
below_table = ''
if len(dataframes)>1:
    if zero:
        below_table = dataframes[1]
        print('---------')
if len(dataframes)>2:    
    if one:
        below_table = dataframes[2]
        print('---------')
        
if len(dataframes)>3:
    if two:
        below_table = dataframes[3]
        print('---------')
if len(dataframes)>3:
    if three:
        below_table = dataframes[4]
        print('---------')

---------


In [880]:
below_table

,Forma de Pago,Vto.,Euros
0,TRANSFERENCIA,09/01/2021,"561,47"


In [881]:
if len(this_data)>4:
    this_data.columns = this_data.columns.str.replace('SELECTED  ', '')
    this_data.columns = this_data.columns.str.replace('SELECTED ', '')
    this_data.columns = this_data.columns.str.replace('SELECTED', '')
    this_data.columns = this_data.columns.str.replace('NOT_', '')
    this_data.columns = this_data.columns.str.replace('NOT_SELECTED  ', '')
    this_data.columns = this_data.columns.str.replace('NOT_SELECTED', '')
    big = this_data.columns[0]
    cont = this_data.columns[2]
    lot = this_data.columns[1]
    prec = this_data.columns[3]
    cant = this_data.columns[-5]
    imp = this_data.columns[-2]
    importe = this_data.columns[-3]
    third = this_data.columns[-4]
    this_data = this_data.loc[(this_data[big] != "Número de pedido") & (this_data[big] != "*** Número de pedido") & (this_data[big]!="*** Número de") & (this_data[big]!="*** Número")]
    this_data = this_data.loc[(this_data[big] != "Entregado a: TABERNA") & (this_data[big] !="Entregado a: TASCA") & (this_data[big]!="Entregado a: TABERNA OSUNA")
          & (this_data[big]!= "Entregado a: TABERNA osuna")& (this_data[big]!="Fin de número") ]
    this_data = this_data.loc[(this_data[imp]!= "Spain Fecha:") & (this_data[imp]!= "Spain Fecha:0") & (this_data[imp]!= "Spain Fecha:1")]
    this_data = this_data.loc[this_data[cont] !="LOT:"]
    this_data = this_data.loc[this_data[cant] !="Total"]
    this_data = this_data.loc[this_data[cant]!="LA LAGUNA" ]
    this_data = this_data.replace('',np.nan)
    this_data = this_data.dropna(axis=0, thresh=3, how="any")
    limitPer = len(this_data) * .100
    this_data = this_data.dropna(thresh=limitPer, axis=1)
    this_data = this_data.replace(np.nan,'0')

In [882]:
data = this_data
big = this_data

In [883]:
#for i in this_data.columns:
#    if i == '':
#        this_data.drop(i,  inplace=True)
if len(big.columns)==3:
    print('Hi')

In [884]:
this_data.head()

,Cantidad Envase,Artículo,I.V.A.,Euro Uni.,Dto.,Euros Línea,Neto Kg/Un Aproximado
1,"5,00 1Kg Caja",Alfalfa CAPITANA R1,0+0,"6,500","0,00","32,50","6,500"
2,"10,00 Bols Eco",Calabaza Butternut ECO,0+0,"1,000","30,00","7,00","0,700"
3,"10,00 Bols Eco",Calabaza Mallorca ECO,0+0,"1,000","30,00","7,00","0,700"
4,"10,00 Bols Eco",Lechuga Cuatro estaciones - ECO,0+0,"1,000","30,00","7,00","0,700"
5,"10,00 Bols Eco",Pepino Marketmore 70 - ECO,0+0,"1,000","30,00","7,00","0,700"


In [885]:
# for  second column
this_data["second_count"] = this_data.iloc[:, 1].str.count("\D")
this_data["second_length"] = this_data.iloc[:, 1].str.len()
this_data["second_digit_percentage"] = this_data["second_count"] / this_data["second_length"] * 100



In [886]:
# for third column
this_data["third_count"] = this_data.iloc[:, 2].str.count("\D")
this_data["third_length"] = this_data.iloc[:, 2].str.len()
this_data["third_digit_percentage"] = this_data["third_count"] / this_data["third_length"] * 100



In [887]:
len(this_data.columns)

13

In [888]:
this_data.head()

,Cantidad Envase,Artículo,I.V.A.,Euro Uni.,Dto.,Euros Línea,Neto Kg/Un Aproximado,second_count,second_length,second_digit_percentage,third_count,third_length,third_digit_percentage
1,"5,00 1Kg Caja",Alfalfa CAPITANA R1,0+0,"6,500","0,00","32,50","6,500",18,19,94.736842,1,3,33.333333
2,"10,00 Bols Eco",Calabaza Butternut ECO,0+0,"1,000","30,00","7,00","0,700",22,22,100.000000,1,3,33.333333
3,"10,00 Bols Eco",Calabaza Mallorca ECO,0+0,"1,000","30,00","7,00","0,700",21,21,100.000000,1,3,33.333333
4,"10,00 Bols Eco",Lechuga Cuatro estaciones - ECO,0+0,"1,000","30,00","7,00","0,700",31,31,100.000000,1,3,33.333333
5,"10,00 Bols Eco",Pepino Marketmore 70 - ECO,0+0,"1,000","30,00","7,00","0,700",24,26,92.307692,1,3,33.333333


In [889]:
# for  forth column
if len(data.columns)==3:
    this_data["forth_count"] = this_data.iloc[:, 3].str.count("\D")
    this_data["forth_length"] = this_data.iloc[:, 3].str.len()
    this_data["forth_digit_percentage"] = this_data["forth_count"] / this_data["forth_length"] * 100
    dftwo = True



In [890]:
this_data.fillna(0)

,Cantidad Envase,Artículo,I.V.A.,Euro Uni.,Dto.,Euros Línea,Neto Kg/Un Aproximado,second_count,second_length,second_digit_percentage,third_count,third_length,third_digit_percentage
1,"5,00 1Kg Caja",Alfalfa CAPITANA R1,0+0,"6,500","0,00","32,50","6,500",18,19,94.736842,1,3,33.333333
2,"10,00 Bols Eco",Calabaza Butternut ECO,0+0,"1,000","30,00","7,00","0,700",22,22,100.000000,1,3,33.333333
3,"10,00 Bols Eco",Calabaza Mallorca ECO,0+0,"1,000","30,00","7,00","0,700",21,21,100.000000,1,3,33.333333
4,"10,00 Bols Eco",Lechuga Cuatro estaciones - ECO,0+0,"1,000","30,00","7,00","0,700",31,31,100.000000,1,3,33.333333
5,"10,00 Bols Eco",Pepino Marketmore 70 - ECO,0+0,"1,000","30,00","7,00","0,700",24,26,92.307692,1,3,33.333333
6,"10,00 Bols Eco",Perejil Común 2 (hoja grande) ECO,0+0,"1,000","30,00","7,00","0,700",32,33,96.969697,1,3,33.333333
7,"10,00 Bols Eco",Pimiento Largo de Reus - ECO,0+0,"1,000","30,00","7,00","0,700",28,28,100.000000,1,3,33.333333
8,"10,00 Bols Eco",Pimiento Dulce Italiano - ECO,0+0,"1,000","30,00","7,00","0,700",29,29,100.000000,1,3,33.333333
9,"10,00 Bols Eco",Rabanito medio largo rojo P.BLANCA -,0+0,"1,000","30,00","7,00","0,700",36,36,100.000000,1,3,33.333333
10,"10,00 Bols Eco",Zananhoria Nantesa ECO,0+0,"1,000","30,00","7,00","0,700",22,22,100.000000,1,3,33.333333


In [891]:
df = this_data.query("second_digit_percentage>30.0")
df1 = this_data.query("third_digit_percentage>30.0")
if dftwo:
    df2.fillna(0)
    df2 = this_data.query("forth_digit_percentage>60.0")


In [892]:
df.shape

(31, 13)

In [893]:
#df2 = this_data.query("second_digit_percentage>30.0")
this_data.shape

(31, 13)

In [894]:
this_data.head()

,Cantidad Envase,Artículo,I.V.A.,Euro Uni.,Dto.,Euros Línea,Neto Kg/Un Aproximado,second_count,second_length,second_digit_percentage,third_count,third_length,third_digit_percentage
1,"5,00 1Kg Caja",Alfalfa CAPITANA R1,0+0,"6,500","0,00","32,50","6,500",18,19,94.736842,1,3,33.333333
2,"10,00 Bols Eco",Calabaza Butternut ECO,0+0,"1,000","30,00","7,00","0,700",22,22,100.000000,1,3,33.333333
3,"10,00 Bols Eco",Calabaza Mallorca ECO,0+0,"1,000","30,00","7,00","0,700",21,21,100.000000,1,3,33.333333
4,"10,00 Bols Eco",Lechuga Cuatro estaciones - ECO,0+0,"1,000","30,00","7,00","0,700",31,31,100.000000,1,3,33.333333
5,"10,00 Bols Eco",Pepino Marketmore 70 - ECO,0+0,"1,000","30,00","7,00","0,700",24,26,92.307692,1,3,33.333333


In [895]:
desc = ''
if len(df.columns) == len(this_data.columns):
    desc = df.iloc[:,1]
    print(' I got in ')
elif len(df1.columns)== len(this_data.columns):
    desc = df1.iloc[:,2]
    print('I got it ')
elif len(df2.columns)== len(this_data.columns):
    desc = df2.iloc[:,3]
    print('I got it2 ')

 I got in 


In [896]:
this_data.head()

,Cantidad Envase,Artículo,I.V.A.,Euro Uni.,Dto.,Euros Línea,Neto Kg/Un Aproximado,second_count,second_length,second_digit_percentage,third_count,third_length,third_digit_percentage
1,"5,00 1Kg Caja",Alfalfa CAPITANA R1,0+0,"6,500","0,00","32,50","6,500",18,19,94.736842,1,3,33.333333
2,"10,00 Bols Eco",Calabaza Butternut ECO,0+0,"1,000","30,00","7,00","0,700",22,22,100.000000,1,3,33.333333
3,"10,00 Bols Eco",Calabaza Mallorca ECO,0+0,"1,000","30,00","7,00","0,700",21,21,100.000000,1,3,33.333333
4,"10,00 Bols Eco",Lechuga Cuatro estaciones - ECO,0+0,"1,000","30,00","7,00","0,700",31,31,100.000000,1,3,33.333333
5,"10,00 Bols Eco",Pepino Marketmore 70 - ECO,0+0,"1,000","30,00","7,00","0,700",24,26,92.307692,1,3,33.333333


In [897]:
description_list = ["Descripción","Descrip. artículo","DESCRIPCION","Description articulo","Description","DESCRIPCIÓN ARTICULO","Unidades Descripción"]
if len(desc) == 0:
    print('Empty')
    for i in ((description_list)):
        if i in this_data.columns:
            print(i)
            dfff = this_data[i]    
            break
       

In [898]:
desc

1                         Alfalfa CAPITANA R1
2                      Calabaza Butternut ECO
3                       Calabaza Mallorca ECO
4             Lechuga Cuatro estaciones - ECO
5                  Pepino Marketmore 70 - ECO
6           Perejil Común 2 (hoja grande) ECO
7                Pimiento Largo de Reus - ECO
8               Pimiento Dulce Italiano - ECO
9        Rabanito medio largo rojo P.BLANCA -
10                     Zananhoria Nantesa ECO
11        Albahaca Genovesa hoja grande - ECO
12    Manzanilla (Matricaria chamomilla) - EC
13                                  Menta ECO
14          Oregano - Origanum majorana - ECO
15                     Fert. Universal 1250ml
16          Fert. Ecoyerba Crecimiento 1250ml
17            Fert. Ecoyerba Floracion 1250ml
18           Fert. Universal Pellet Eco 2'5kg
19                      Fert. Universal 400ml
20                 Fert. Plantas Verdes 400ml
21                      Triple Accion Quimico
22          Insecticida Polivalent

In [899]:
if len(desc)!=0:
    desc_data = pd.DataFrame(desc)
    desc_data.head()
elif len(dfff)!=0:
    desc_data = pd.DataFrame(dfff)
    desc_data.head()

In [900]:
desc_data.head()

,Artículo
1,Alfalfa CAPITANA R1
2,Calabaza Butternut ECO
3,Calabaza Mallorca ECO
4,Lechuga Cuatro estaciones - ECO
5,Pepino Marketmore 70 - ECO


In [901]:
this_data

,Cantidad Envase,Artículo,I.V.A.,Euro Uni.,Dto.,Euros Línea,Neto Kg/Un Aproximado,second_count,second_length,second_digit_percentage,third_count,third_length,third_digit_percentage
1,"5,00 1Kg Caja",Alfalfa CAPITANA R1,0+0,"6,500","0,00","32,50","6,500",18,19,94.736842,1,3,33.333333
2,"10,00 Bols Eco",Calabaza Butternut ECO,0+0,"1,000","30,00","7,00","0,700",22,22,100.000000,1,3,33.333333
3,"10,00 Bols Eco",Calabaza Mallorca ECO,0+0,"1,000","30,00","7,00","0,700",21,21,100.000000,1,3,33.333333
4,"10,00 Bols Eco",Lechuga Cuatro estaciones - ECO,0+0,"1,000","30,00","7,00","0,700",31,31,100.000000,1,3,33.333333
5,"10,00 Bols Eco",Pepino Marketmore 70 - ECO,0+0,"1,000","30,00","7,00","0,700",24,26,92.307692,1,3,33.333333
6,"10,00 Bols Eco",Perejil Común 2 (hoja grande) ECO,0+0,"1,000","30,00","7,00","0,700",32,33,96.969697,1,3,33.333333
7,"10,00 Bols Eco",Pimiento Largo de Reus - ECO,0+0,"1,000","30,00","7,00","0,700",28,28,100.000000,1,3,33.333333
8,"10,00 Bols Eco",Pimiento Dulce Italiano - ECO,0+0,"1,000","30,00","7,00","0,700",29,29,100.000000,1,3,33.333333
9,"10,00 Bols Eco",Rabanito medio largo rojo P.BLANCA -,0+0,"1,000","30,00","7,00","0,700",36,36,100.000000,1,3,33.333333
10,"10,00 Bols Eco",Zananhoria Nantesa ECO,0+0,"1,000","30,00","7,00","0,700",22,22,100.000000,1,3,33.333333


In [902]:
len(data.columns)

13

In [903]:
this_data

,Cantidad Envase,Artículo,I.V.A.,Euro Uni.,Dto.,Euros Línea,Neto Kg/Un Aproximado,second_count,second_length,second_digit_percentage,third_count,third_length,third_digit_percentage
1,"5,00 1Kg Caja",Alfalfa CAPITANA R1,0+0,"6,500","0,00","32,50","6,500",18,19,94.736842,1,3,33.333333
2,"10,00 Bols Eco",Calabaza Butternut ECO,0+0,"1,000","30,00","7,00","0,700",22,22,100.000000,1,3,33.333333
3,"10,00 Bols Eco",Calabaza Mallorca ECO,0+0,"1,000","30,00","7,00","0,700",21,21,100.000000,1,3,33.333333
4,"10,00 Bols Eco",Lechuga Cuatro estaciones - ECO,0+0,"1,000","30,00","7,00","0,700",31,31,100.000000,1,3,33.333333
5,"10,00 Bols Eco",Pepino Marketmore 70 - ECO,0+0,"1,000","30,00","7,00","0,700",24,26,92.307692,1,3,33.333333
6,"10,00 Bols Eco",Perejil Común 2 (hoja grande) ECO,0+0,"1,000","30,00","7,00","0,700",32,33,96.969697,1,3,33.333333
7,"10,00 Bols Eco",Pimiento Largo de Reus - ECO,0+0,"1,000","30,00","7,00","0,700",28,28,100.000000,1,3,33.333333
8,"10,00 Bols Eco",Pimiento Dulce Italiano - ECO,0+0,"1,000","30,00","7,00","0,700",29,29,100.000000,1,3,33.333333
9,"10,00 Bols Eco",Rabanito medio largo rojo P.BLANCA -,0+0,"1,000","30,00","7,00","0,700",36,36,100.000000,1,3,33.333333
10,"10,00 Bols Eco",Zananhoria Nantesa ECO,0+0,"1,000","30,00","7,00","0,700",22,22,100.000000,1,3,33.333333


In [904]:
code  = this_data.iloc[:, 0]
#codingo = this_data[code]
codingo_data  =  pd.DataFrame(code)
codingo_data


,Cantidad Envase
1,"5,00 1Kg Caja"
2,"10,00 Bols Eco"
3,"10,00 Bols Eco"
4,"10,00 Bols Eco"
5,"10,00 Bols Eco"
6,"10,00 Bols Eco"
7,"10,00 Bols Eco"
8,"10,00 Bols Eco"
9,"10,00 Bols Eco"
10,"10,00 Bols Eco"


In [905]:
prec = []
str1 = "Precio"
str2 = "Base I"
str3 = "PVP"
str4 = 'Importe'
str5 = 'Euro Uni.'
str6 = "IMP"
price =''
for i in this_data.columns:
    ratio1 = fuzz.partial_ratio(str1.lower(),str(i).lower())
    if ratio1>=75:
        print(i)
        prec.append(i)
        price = i
        print("Im condition 1")
    ratio2 = fuzz.partial_ratio(str2.lower(),str(i).lower())
    if ratio2>=75:
        print(i)
        prec.append(i)
        price =i
        print("Im condition 2")
    ratio3 = fuzz.partial_ratio(str3.lower(),str(i).lower())
    if ratio3>=75:
        print("Im condition 3")
        print(i)
        prec.append(i)
        price =i
    ratio4 = fuzz.partial_ratio(str4.lower(),str(i).lower())
    if ratio4>=75:
        print("Im condition 4")
        print(i)
        prec.append(i)
        price =i
    ratio5 = fuzz.partial_ratio(str5.lower(),str(i).lower())
    if ratio5>=75:
        print("Im condition 5")
        print(i)
        prec.append(i)
        price =i
    ratio6 = fuzz.partial_ratio(str6.lower(),str(i).lower())
    if ratio6>=75:
        print("Im condition 6")
        print(i)
        prec.append(i)
        price =i
if len(prec)>1:
    price = prec[0]
else:
    pass

Im condition 5
Euro Uni.


In [906]:
if price !='':
    price_col = this_data[price]
    price_data = pd.DataFrame(price_col,columns=[price])
    price_data
else:
    print("Price Is Empty")

In [907]:
str1 = "DTO"
str2 = "Descuento"
str3 = "DCTO"
str4 = "Desc"
disc = ''
for i in this_data.columns:
    ratio1 = fuzz.ratio(str1.lower(),str(i).lower())
    ratio2 = fuzz.ratio(str2.lower(),str(i).lower())
    ratio3 = fuzz.ratio(str3.lower(),str(i).lower())
    ratio4 = fuzz.ratio(str4.lower(),str(i).lower())
    if ratio1>=75:
        print(i)
        disc = i
        print("Im condition 1")
    
    elif ratio2>=75:
        print(i)
        disc = i
        print("Im condition 2")
    
    elif ratio3>=75:
        print(i)
        disc = i
        print("Im condition 3")
    
    elif ratio4>=75:
        print(i)
        disc = i
        print("Im condition 4")
    
print(disc)

Dto.
Im condition 1
Dto.


In [908]:
if disc!='':
    disc_col = this_data[disc]
    disc_data = pd.DataFrame(disc_col,columns=[disc])
    disc_data

In [909]:
str1 = "Cantidad"
str2 = "Cant"
str3 = "Unid"
str4 = "Unidades"
str5 = "Uds"
str6 = "Peso"
cat = ''
for i in this_data.columns:
    ratio1 = fuzz.ratio(str1.lower(),str(i).lower())
    if ratio1>=75:
        print(i)
        cat = i
        print("Im condition 1")
    ratio2 = fuzz.ratio(str2.lower(),str(i).lower())
    if ratio2>=75:
        print(i)
        cat =i
        print("Im condition 2")
    ratio3 = fuzz.ratio(str3.lower(),str(i).lower())
    if ratio3>=75:
        print("Im condition 3")
        print(i)
        cat =i
    ratio4 = fuzz.ratio(str4.lower(),str(i).lower())
    if ratio4>=75:
        print("Im condition 4")
        print(i)
        cat =i
    ratio5 = fuzz.ratio(str5.lower(),str(i).lower())
    if ratio5>=75:
        print("Im condition 5")
        print(i)
        cat =i
    ratio6 = fuzz.ratio(str6.lower(),str(i).lower())
    if ratio6>=75:
        print("Im condition 6")
        print(i)
        cat =i

In [910]:
if cat!='':
    Cantidad = this_data[cat]
    Cantidad_data = pd.DataFrame(Cantidad,columns=[cat])
    Cantidad_data
else:
    print("Quantity is Empty")
Cantidad_data

Quantity is Empty


,Cant
0,2
1,1
2,2
3,1
4,1
5,1
6,1
7,1
8,1
9,1


In [911]:
str1 = 'IMPUESTO'
imposto = ''
for i in this_data.columns:
    ratio1 = fuzz.partial_ratio(str1.lower(),str(i).lower())
    if ratio1>=75:
        print(i)
        imposto = i
print(imposto)

In [912]:
if imposto!='':
    imposto_col = this_data[imposto]
    imposto_data = pd.DataFrame(imposto_col,columns=[imposto])
    imposto_data

In [913]:
str1 = 'Tasa'
str2 = 'TIPO'
tasa = ''
for i in this_data.columns:
    ratio1 = fuzz.ratio(str1.lower(),str(i).lower())
    if ratio1>=75:
        print(i)
        tasa = i
    ratio2 = fuzz.ratio(str2.lower(),str(i).lower())
    if ratio2>=75:
        print(i)
        tasa = i
print(tasa)

In [914]:
if tasa!='':
    tasa_col = this_data[tasa]
    tasa_data = pd.DataFrame(tasa_col,columns=[tasa])
    tasa_data

In [915]:
#codingo_data
#dataf
#Cantidad_data
#price_data

In [916]:
if disc =='' and imposto =='' and tasa =='' and cat=='' and price == '':
    df2 = pd.concat([codingo_data,desc_data],axis=1, join='inner')
    df2['Price'] = 0
    df2['Quantity'] = 0
    df2['Discount'] = 0
    df2['IMPUESTO']=0
    df2['Tasa']=0
   
   

elif disc!='' and imposto =='' and tasa =='':
    disc_col = this_data[disc]
    disc_data = pd.DataFrame(disc_col,columns=[disc])
    df2 = pd.concat([codingo_data,desc_data,Cantidad_data,price_data,disc_data],axis=1, join='inner')
    df2['IMPUESTO']=0
    df2['Tasa']=0
    
elif disc!='' and imposto!='' and tasa=='':
    df2 = pd.concat([codingo_data,desc_data,Cantidad_data,price_data,disc_data,imposto_data],axis=1, join='inner')
    df2['Tasa']=0
    
elif disc!='' and imposto=='' and tasa!='':
    df2 = pd.concat([codingo_data,desc_data,Cantidad_data,price_data,disc_data,tasa_data],axis=1, join='inner')
    df2['IMPUESTO']=0
    
elif disc=='' and imposto!='' and tasa!='':
    df2 = pd.concat([codingo_data,desc_data,Cantidad_data,price_data,imposto_data,tasa_data],axis=1, join='inner')
    df2['Discount']=0
    
elif disc=='' and imposto=='' and tasa!='':
    df2 = pd.concat([codingo_data,desc_data,Cantidad_data,price_data,tasa_data],axis=1, join='inner')
    df2['Discount']=0
    df2['IMPUESTO']=0
    
elif disc=='' and imposto!='' and tasa=='':
    df2 = pd.concat([codingo_data,desc_data,Cantidad_data,price_data,imposto_data],axis=1, join='inner')
    df2['Discount']=0
    df2['Tasa']=0

elif disc=='' and imposto=='' and tasa=='':
    df2 = pd.concat([codingo_data,desc_data,Cantidad_data,price_data],axis=1, join='inner')
    df2['Discount']=0
    df2['IMPUESTO']=0
    df2['Tasa']=0

In [917]:
df2

,Cantidad Envase,Artículo,Cant,Euro Uni.,Dto.,IMPUESTO,Tasa
1,"5,00 1Kg Caja",Alfalfa CAPITANA R1,1,"6,500","0,00",0,0
2,"10,00 Bols Eco",Calabaza Butternut ECO,2,"1,000","30,00",0,0
3,"10,00 Bols Eco",Calabaza Mallorca ECO,1,"1,000","30,00",0,0
4,"10,00 Bols Eco",Lechuga Cuatro estaciones - ECO,1,"1,000","30,00",0,0
5,"10,00 Bols Eco",Pepino Marketmore 70 - ECO,1,"1,000","30,00",0,0
6,"10,00 Bols Eco",Perejil Común 2 (hoja grande) ECO,1,"1,000","30,00",0,0
7,"10,00 Bols Eco",Pimiento Largo de Reus - ECO,1,"1,000","30,00",0,0
8,"10,00 Bols Eco",Pimiento Dulce Italiano - ECO,1,"1,000","30,00",0,0
9,"10,00 Bols Eco",Rabanito medio largo rojo P.BLANCA -,1,"1,000","30,00",0,0
10,"10,00 Bols Eco",Zananhoria Nantesa ECO,1,"1,000","30,00",0,0


In [347]:
if disc =='' and imposto=='' and tasa=='' and code!='':
    df2 = pd.concat([codingo_data,dataf,price_data,Cantidad_data],axis=1, join='inner')
    df2['Discount'] = 0
    df2['IMPUESTO']=0
    df2['Tasa']=0
    
elif code=='' and disc =='' and imposto=='' and tasa=='':
    df2 = pd.concat([dataf,price_data,Cantidad_data],axis=1, join='inner')
    df2['Codigo'] = 0
    df2['Discount'] = 0
    df2['IMPUESTO']=0
    df2['Tasa']=0
    
elif code=='' and disc!='' and imposto=='' and tasa=='':
    discount = this_data[disc]
    discount_data = pd.DataFrame(discount,columns=[disc])
    df2 = pd.concat([dataf,price_data,Cantidad_data,discount_data],axis=1, join='inner')
    df2['Codigo'] = 0
    df2['IMPUESTO']=0
    df2['Tasa']=0
    
elif disc !='' and imposto=='' and tasa=='' and code!='':
    discount = this_data[disc]
    discount_data = pd.DataFrame(discount,columns=[disc])
    df2 = pd.concat([codingo_data,dataf,price_data,Cantidad_data,discount_data],axis=1, join='inner')
    df2['IMPUESTO']=0
    df2['Tasa']=0
    
elif disc !='' and imposto!='' and tasa=='' and code!='':
    discount = this_data[disc]
    discount_data = pd.DataFrame(discount,columns=[disc])
    impoto = this_data[imposto]
    imposto_data = pd.DataFrame(impoto,columns[imposto])
    df2 = pd.concat([codingo_data,dataf,price_data,Cantidad_data,discount_data,imposto_data],axis=1, join='inner')
    df2['Tasa']=0
    
elif disc =='' and imposto!='' and tasa!='' and code!='':
    impoto = this_data[imposto]
    imposto_data = pd.DataFrame(impoto,columns[imposto])
    tas = this_data[tasa]
    tasa_data = pd.DataFrame(tas,columns[tasa])
    df2 = pd.concat([codingo_data,dataf,price_data,Cantidad_data,imposto_data,tasa_data],axis=1, join='inner')
    df2['Discount']=0
    
else:
    discount = this_data[disc]
    discount_data = pd.DataFrame(discount,columns=[disc])
    impota = this_data[imposto]
    impota_data = pd.DataFrame(impota,columns=[imposto])
    df2 = pd.concat([codingo_data,dataf,price_data,Cantidad_data,discount_data,impota_data],axis=1, join='inner')

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [161]:
#df = pd.DataFrame([dataf,price_data], columns=['Description','Price'])
#df2 = pd.concat([dataf,price_data,Cantidad_data,discount_data],axis=1, join='inner')


DCTO

In [61]:
df2

,ARTÍCULO,ARTÍCULO,PRECIO UD.,CANTIDAD,DTO.,IMPUESTO,Tasa
0,0516,0516,"5,25","8,00",,0,0


In [134]:
#data['Vat Number']= data['Vat Number'].str.replace(',', '')
df2.columns.values[0] = "Codigo"
df2.columns.values[1] = "Description"
df2.columns.values[2] = "Precio"
df2.columns.values[3] = "Cantidad"
df2.columns.values[4] = "Discount"

df2


,Codigo,Description,Precio,Cantidad,Discount,IMPUESTO,Tasa
0,CUOTA FEMETE JULIO 2020,"49,52 €","49,52 €",0,0,0,0
1,,SUBTOTAL,SUBTOTAL,0,0,0,0
2,,IGIC(%)(1],IGIC(%)(1],0,0,0,0
3,,TOTAL,TOTAL,0,0,0,0


In [80]:
df2["Precio"]=df2["Precio"].str.replace(',','.')
df2["Cantidad"]=df2["Cantidad"].str.replace(',','.')
df2["Precio"]=pd.to_numeric(df2["Precio"])

df2
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 31 entries, 1 to 31
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Codigo       31 non-null     object 
 1   Description  31 non-null     object 
 2   Precio       31 non-null     float64
 3   Cantidad     31 non-null     object 
 4   Discount     31 non-null     int64  
 5   IMPUESTO     31 non-null     int64  
 6   Tasa         31 non-null     int64  
dtypes: float64(1), int64(3), object(3)
memory usage: 1.9+ KB


In [81]:
df2

,Codigo,Description,Precio,Cantidad,Discount,IMPUESTO,Tasa
1,Alfalfa CAPITANA R1,"6,500",6.500,0.00,0,0,0
2,Calabaza Butternut ECO,"1,000",1.000,30.00,0,0,0
3,Calabaza Mallorca ECO,"1,000",1.000,30.00,0,0,0
4,Lechuga Cuatro estaciones - ECO,"1,000",1.000,30.00,0,0,0
5,Pepino Marketmore 70 - ECO,"1,000",1.000,30.00,0,0,0
6,Perejil Común 2 (hoja grande) ECO,"1,000",1.000,30.00,0,0,0
7,Pimiento Largo de Reus - ECO,"1,000",1.000,30.00,0,0,0
8,Pimiento Dulce Italiano - ECO,"1,000",1.000,30.00,0,0,0
9,Rabanito medio largo rojo P.BLANCA -,"1,000",1.000,30.00,0,0,0
10,Zananhoria Nantesa ECO,"1,000",1.000,30.00,0,0,0


In [86]:
import mysql.connector

#establishing the connection
conn = mysql.connector.connect(
   user='root', password='admin', host='localhost', database='sakila')
cursor = conn.cursor()

# Preparing SQL query to INSERT a record into the database.
sql = """INSERT INTO Dataa(
   vat_number,total,date_,document_type,invoice_number)
   VALUES (%s, %s, %s, %s, %s)"""
val = (dat["Vat Number"],dat["Total"],dat["Date"],dat["Document_Type"],dat["Invoice_Number"])
# Executing the SQL command
cursor.execute(sql,val)
conn.commit()
conn.close()


In [97]:
mycursor = con.cursor()
mycursor.execute('select * from dataa')
print(mycursor)
for tb in mycursor:
    print(tb)
#data = pd.DataFrame(tb)

CMySQLCursor: select * from dataa
('B76744861', 'www.gmfood.es,', '03-02-2020', 'Albaran', '200302180004999ALBARAN')
('B76744861', '52,27', '05/12/2020', 'Factura', '')
('B35129584', '52,27', '05/12/2020', 'Factura', '')
('B38947636', '250,82', '27/05/2022', 'Factura', '0/0(006)0052/(2022)147042')
('B38947636', '250,82', '27/05/2022', 'Factura', '0/0(006)0052/(2022)147042')
('B38947636', '261,26', '30/05/2022', 'Factura', '0/0(006)0052/(2022)150028')


In [94]:
import mysql.connector as c
import pandas as pd
con = c.connect(host='localhost',user='root',passwd="admin",database='sakila')

In [ ]:
insert into data_extraction(id_supplier,id_company,vat_client,vat_supplier,date,invoicenumber,document_type,file,data,total)
values(1,1,'B38947636','B38947636','2022-04-09','12456','Factura','Hi_This_is_Invoice','Data','100')

# Code for Server To Extract Table

In [10]:
import boto3
import io
from PIL import Image
import pandas as pd
import mysql.connector
import numpy as np
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

def read_image(Ima):
    im = Image.open(Ima)
    buffered = io.BytesIO()
    im.save(buffered, format='PNG')
    client = boto3.client('textract', region_name='us-east-2')
    response = client.analyze_document(
        Document={'Bytes': buffered.getvalue()},
        FeatureTypes=['TABLES'])

    def map_blocks(blocks, block_type):
        return {
            block['Id']: block
            for block in blocks
            if block['BlockType'] == block_type
        }

    blocks = response['Blocks']
    tables = map_blocks(blocks, 'TABLE')
    cells = map_blocks(blocks, 'CELL')
    words = map_blocks(blocks, 'WORD')
    selections = map_blocks(blocks, 'SELECTION_ELEMENT')

    def get_children_ids(block):
        for rels in block.get('Relationships', []):
            if rels['Type'] == 'CHILD':
                yield from rels['Ids']

    dataframes = []

    for table in tables.values():

        # Determine all the cells that belong to this table
        table_cells = [cells[cell_id] for cell_id in get_children_ids(table)]

        # Determine the table's number of rows and columns
        n_rows = max(cell['RowIndex'] for cell in table_cells)
        n_cols = max(cell['ColumnIndex'] for cell in table_cells)
        content = [[None for _ in range(n_cols)] for _ in range(n_rows)]

        # Fill in each cell
        for cell in table_cells:
            cell_contents = [
                words[child_id]['Text']
                if child_id in words
                else selections[child_id]['SelectionStatus']
                for child_id in get_children_ids(cell)
            ]
            i = cell['RowIndex'] - 1
            j = cell['ColumnIndex'] - 1
            content[i][j] = ' '.join(cell_contents)

    # We assume that the first row corresponds to the column names
        dataframe = pd.DataFrame(content[1:], columns=content[0])
        dataframes.append(dataframe)
    this_data = None
    str1 = 'DESCRIPCIÓN'
    str2 = "Descrip. artículo"
    str3 = "Artículo/ Cod.cliente"
    str4 = 'DESCRIPCIÓN:'
    str5 = "CONCEPTO"
    str6 = "UDS"
    str7 = "Concepto"
    str8 = "Descripción"
    str9 = "DESCRIPCION"
    str10 = "Description articulo"
    str11 = "Description"
    str12 = "DESCRIPCIÓN ARTICULO"
    str13 = "Unidades Descripción"
    str14 = "PRODUCTO"
    str15 = "Articulo"
    str16 = 'Denominación /Referencia'
    if "ARTÍCULO" in dataframes[0].columns or "MM Num. articulo" in dataframes[0].columns or "MM Num. artículo" in dataframes[0].columns or 'CÓDIGO' in dataframes[0].columns or 'MM Num.' in dataframes[0].columns or str3 in dataframes[0].columns or str4 in dataframes[0].columns or str5 in dataframes[0].columns or str6 in dataframes[0].columns or str7 in dataframes[0].columns or str8 in dataframes[0].columns or str9 in dataframes[0].columns or str10 in dataframes[0].columns or str11 in dataframes[0].columns or str12 in dataframes[0].columns or str13 in dataframes[0].columns or str14 in dataframes[0].columns or str15 in dataframes[0].columns or str16 in dataframes[0].columns:
        this_data = pd.DataFrame(dataframes[0])

    elif "ARTÍCULO" in dataframes[1].columns or "MM Num. articulo" in dataframes[1].columns or "MM Num. artículo" in dataframes[1].columns or 'CÓDIGO' in dataframes[1].columns or 'MM Num.' in dataframes[1].columns or str3 in dataframes[1].columns or str4 in dataframes[1].columns or str5 in dataframes[1].columns or str6 in dataframes[1].columns or str7 in dataframes[1].columns or str8 in dataframes[1].columns or str9 in dataframes[1].columns or str10 in dataframes[1].columns or str11 in dataframes[1].columns or str12 in dataframes[1].columns or str13 in dataframes[1].columns or str14 in dataframes[1].columns or str15 in dataframes[1].columns or str16 in dataframes[1].columns:
        this_data = pd.DataFrame(dataframes[1])

    elif "ARTÍCULO" in dataframes[2].columns or "MM Num. articulo" in dataframes[2].columns or "MM Num. artículo" in dataframes[2].columns or 'CÓDIGO' in dataframes[2].columns or 'MM Num.' in dataframes[2].columns or str3 in dataframes[2].columns or str4 in dataframes[2].columns or str5 in dataframes[2].columns or str6 in dataframes[2].columns or str7 in dataframes[2].columns or str8 in dataframes[2].columns or str9 in dataframes[2].columns or str10 in dataframes[2].columns or str11 in dataframes[2].columns or str12 in dataframes[2].columns or str13 in dataframes[2].columns or str14 in dataframes[2].columns or str15 in dataframes[2].columns or str16 in dataframes[2].columns:
        this_data = pd.DataFrame(dataframes[2])
    if len(dataframes) >3:
        if "ARTÍCULO" in dataframes[3].columns or "MM Num. articulo" in dataframes[3].columns or "MM Num. artículo" in dataframes[3].columns or 'CÓDIGO' in dataframes[3].columns or 'MM Num.' in dataframes[3].columns or str3 in dataframes[3].columns or str4 in dataframes[3].columns or str5 in dataframes[3].columns or str6 in dataframes[3].columns or str7 in dataframes[3].columns or str8 in dataframes[3].columns or str9 in dataframes[3].columns or str10 in dataframes[3].columns or str11 in dataframes[3].columns or str12 in dataframes[3].columns or str13 in dataframes[3].columns or str14 in dataframes[3].columns or str15 in dataframes[3].columns or str16 in dataframes[3].columns:
            this_data = pd.DataFrame(dataframes[3])

    if this_data is None:
        pass
    else:
        if this_data.shape[1] >4:
            this_data.columns = this_data.columns.str.replace('SELECTED  ', '')
            this_data.columns = this_data.columns.str.replace('SELECTED ', '')
            this_data.columns = this_data.columns.str.replace('SELECTED', '')
            this_data.columns = this_data.columns.str.replace('NOT_', '')
            this_data.columns = this_data.columns.str.replace('NOT_SELECTED  ', '')
            this_data.columns = this_data.columns.str.replace('NOT_SELECTED', '')
            big = this_data.columns[0]
            cont = this_data.columns[2]
            lot = this_data.columns[1]
            prec = this_data.columns[3]
            cant = this_data.columns[-5]
            imp = this_data.columns[-2]
            importe = this_data.columns[-3]
            third = this_data.columns[-4]
            this_data = this_data.loc[
            (this_data[big] != "Número de pedido") & (this_data[big] != "*** Número de pedido") & (
                        this_data[big] != "*** Número de") & (this_data[big] != "*** Número")]
            this_data = this_data.loc[
                (this_data[big] != "Entregado a: TABERNA") & (this_data[big] != "Entregado a: TASCA") & (
                        this_data[big] != "Entregado a: TABERNA OSUNA")
            & (this_data[big] != "Entregado a: TABERNA osuna") & (this_data[big] != "Fin de número")]
            this_data = this_data.loc[(this_data[imp] != "Spain Fecha:") & (this_data[imp] != "Spain Fecha:0") & (
                    this_data[imp] != "Spain Fecha:1")]
            this_data = this_data.loc[this_data[cont] != "LOT:"]
            this_data = this_data.loc[this_data[cant] != "Total"]
            this_data = this_data.loc[this_data[cant] != "LA LAGUNA"]
            this_data = this_data.replace('', np.nan)
            this_data = this_data.dropna(axis=0, thresh=3, how="any")
            limitPer = len(this_data) * .100
            this_data = this_data.dropna(thresh=limitPer, axis=1)
            this_data = this_data.replace(np.nan, '0')
            
            str1 = 'CÓDIGO'
            str2 = 'MM Num.'
            str3 = 'ARTÍCULO'
            str4 = 'Código'
            str5= 'Ref.'
            str6= 'REFERENCIA'
            code = ''
            codigo = []
            for i in this_data.columns:
                ratio1 = fuzz.partial_ratio(str1.lower(), str(i).lower())
                if ratio1 >= 75:
                    print(i)
                    code = i
                    codigo.append(i)
                ratio2 = fuzz.partial_ratio(str2.lower(), str(i).lower())
                if ratio2 >= 75:
                    print(i)
                    code = i
                    codigo.append(i)
                ratio3 = fuzz.partial_ratio(str3.lower(), str(i).lower())
                if ratio3 >= 75:
                    print(i)
                    code = i
                    codigo.append(i)
                ratio4 = fuzz.partial_ratio(str4.lower(), str(i).lower())
                if ratio4 >= 75:
                    print(i)
                    code = i
                    codigo.append(i)
                ratio5 = fuzz.partial_ratio(str5.lower(), str(i).lower())
                if ratio5 >= 75:
                    print(i)
                    code = i
                    codigo.append(i)
                ratio6 = fuzz.partial_ratio(str6.lower(), str(i).lower())
                if ratio6 >= 75:
                    print(i)
                    code = i
                    codigo.append(i)
            if len(codigo) > 1:
                code = codigo[0]
            print(code)

            if code == '':
                pass
            else:
                print('--------------------------------------------------------------------------')
                codingo = this_data[code]
                codingo_data = pd.DataFrame(codingo, columns=[code])
                
            str1 = 'DESCRIPCIÓN'
            str2 = "Descrip. artículo"
            str3 = "Artículo/ Cod.cliente"
            str4 = 'DESCRIPCIÓN:'
            str5 = "CONCEPTO"
            str6 = "UDS"
            str7 = "Concepto"
            str8 = "Descripción"
            str9 = "DESCRIPCION"
            str10 = "Description articulo"
            str11 = "Description"
            str12 = "DESCRIPCIÓN ARTICULO"
            str13 = "Unidades Descripción"
            str14 = "PRODUCTO"
            str15 = "Articulo"
            str16 = 'Denominación /Referencia'
            name = ''
            pro = []
            for i in this_data.columns:
                ratio1 = fuzz.partial_ratio(str1.lower(), str(i).lower())
                if ratio1 >= 75:
                    print(i)
                    name = i
                    pro.append(i)

                ratio2 = fuzz.partial_ratio(str2.lower(), str(i).lower())
                if ratio2 >= 75:
                    print(i)
                    name = i
                    pro.append(i)
                ratio3 = fuzz.partial_ratio(str3.lower(), str(i).lower())
                if ratio3 >= 75:
                    print(i)
                    name = i
                    pro.append(i)

                ratio4 = fuzz.partial_ratio(str4.lower(), str(i).lower())
                if ratio4 >= 75:
                    print(i)
                    name = i
                    pro.append(i)

                ratio5 = fuzz.partial_ratio(str5.lower(), str(i).lower())
                if ratio5 >= 75:
                    print(i)
                    name = i
                    pro.append(i)

                ratio6 = fuzz.partial_ratio(str6.lower(), str(i).lower())
                if ratio6 >= 75:
                    print(i)
                    name = i
                    pro.append(i)
                ratio7 = fuzz.partial_ratio(str7.lower(), str(i).lower())
                if ratio7 >= 75:
                    print(i)
                    name = i
                    pro.append(i)

                ratio8 = fuzz.partial_ratio(str8.lower(), str(i).lower())
                if ratio8 >= 75:
                    print(i)
                    name = i
                    pro.append(i)

                ratio9 = fuzz.partial_ratio(str9.lower(), str(i).lower())
                if ratio9 >= 75:
                    print(i)
                    name = i
                    pro.append(i)

                ratio10 = fuzz.partial_ratio(str10.lower(), str(i).lower())
                if ratio10 >= 75:
                    print(i)
                    name = i
                    pro.append(i)

                ratio11 = fuzz.partial_ratio(str11.lower(), str(i).lower())
                if ratio11 >= 75:
                    print(i)
                    name = i
                    pro.append(i)

                ratio12 = fuzz.partial_ratio(str12.lower(), str(i).lower())
                if ratio12 >= 75:
                    print(i)
                    name = i
                    pro.append(i)

                ratio13 = fuzz.partial_ratio(str13.lower(), str(i).lower())
                if ratio13 >= 75:
                    print(i)
                    name = i
                    pro.append(i)

                ratio14 = fuzz.partial_ratio(str14.lower(), str(i).lower())
                if ratio14 >= 75:
                    print(i)
                    name = i
                    pro.append(i)

                ratio15 = fuzz.partial_ratio(str15.lower(), str(i).lower())
                if ratio15 >= 75:
                    print(i)
                    name = i
                    pro.append(i)

                ratio16 = fuzz.partial_ratio(str16.lower(), str(i).lower())
                if ratio16 >= 75:
                    print(i)
                    name = i
                    pro.append(i)
            print(len(pro))
            if len(pro) > 1:
                name = pro[0]
            if name == '':
                pass
            else:
                this = this_data[name]
                dataf = pd.DataFrame(this, columns=[name])
            
            prec = []
            str1 = "Precio"
            str2 = "Base I"
            str3 = "PVP"
            str4 = 'Importe'
            str5 = 'Euro Uni.'
            str6 = "IMP"
            price = ''
            for i in this_data.columns:
                ratio1 = fuzz.partial_ratio(str1.lower(), str(i).lower())
                if ratio1 >= 75:
                    print(i)
                    prec.append(i)
                    price = i
                    print("Im condition 1")
                ratio2 = fuzz.partial_ratio(str2.lower(), str(i).lower())
                if ratio2 >= 75:
                    print(i)
                    prec.append(i)
                    price = i
                    print("Im condition 2")
                ratio3 = fuzz.partial_ratio(str3.lower(), str(i).lower())
                if ratio3 >= 75:
                    print("Im condition 3")
                    print(i)
                    prec.append(i)
                    price = i
                ratio4 = fuzz.partial_ratio(str4.lower(), str(i).lower())
                if ratio4 >= 75:
                    print("Im condition 4")
                    print(i)
                    prec.append(i)
                    price = i
                ratio5 = fuzz.partial_ratio(str5.lower(), str(i).lower())
                if ratio5 >= 75:
                    print("Im condition 5")
                    print(i)
                    prec.append(i)
                    price = i
                ratio6 = fuzz.partial_ratio(str6.lower(), str(i).lower())
                if ratio6 >= 75:
                    print("Im condition 6")
                    print(i)
                    prec.append(i)
                    price = i
            if len(prec) > 1:
                price = prec[0]
            else:
                pass
            if price == '':
                pass
            else:
                price_col = this_data[price]
                price_data = pd.DataFrame(price_col, columns=[price])
                
            str1 = "DTO"
            str2 = "Descuento"
            str3 = "DCTO"
            str4 = "Desc"
            disc = ''
            for i in this_data.columns:
                ratio1 = fuzz.ratio(str1.lower(), str(i).lower())
                ratio2 = fuzz.ratio(str2.lower(), str(i).lower())
                ratio3 = fuzz.ratio(str3.lower(), str(i).lower())
                ratio4 = fuzz.ratio(str4.lower(), str(i).lower())
                if ratio1 >= 75:
                    print(i)
                    disc = i
                    print("Im condition 1")

                elif ratio2 >= 75:
                    print(i)
                    disc = i
                    print("Im condition 2")

                elif ratio3 >= 75:
                    print(i)
                    disc = i
                    print("Im condition 3")

                elif ratio4 >= 75:
                    print(i)
                    disc = i
                    print("Im condition 4")

            str1 = "Cantidad"
            str2 = "Cant"
            str3 = "Unid"
            str4 = "Unidades"
            str5 = "Uds"
            str6 = "Peso"
            cat =""
            for i in this_data.columns:
                ratio1 = fuzz.partial_ratio(str1.lower(), str(i).lower())
                if ratio1 >= 75:
                    print(i)
                    cat = i
                    print("Im condition 1")
                ratio2 = fuzz.partial_ratio(str2.lower(), str(i).lower())
                if ratio2 >= 75:
                    print(i)
                    cat = i
                    print("Im condition 2")
                ratio3 = fuzz.partial_ratio(str3.lower(), str(i).lower())
                if ratio3 >= 75:
                    print("Im condition 3")
                    print(i)
                    cat = i
                ratio4 = fuzz.partial_ratio(str4.lower(), str(i).lower())
                if ratio4 >= 75:
                    print("Im condition 4")
                    print(i)
                    cat = i
                ratio5 = fuzz.partial_ratio(str5.lower(), str(i).lower())
                if ratio5 >= 75:
                    print("Im condition 5")
                    print(i)
                    cat = i
                ratio6 = fuzz.partial_ratio(str6.lower(), str(i).lower())
                if ratio6 >= 75:
                    print("Im condition 6")
                    print(i)
                    cat = i
            if cat =='':
                pass
            else:
                Cantidad = this_data[cat]
                Cantidad_data = pd.DataFrame(Cantidad, columns=[cat])
                
            str1 = 'IMPUESTO'
            imposto = ''

            for i in this_data.columns:
                ratio1 = fuzz.partial_ratio(str1.lower(), str(i).lower())
                if ratio1 >= 75:
                    print(i)
                    imposto = i
            print(imposto)

            if imposto != '':
                imposto_col = this_data[imposto]
                imposto_data = pd.DataFrame(imposto_col, columns=[imposto])
                imposto_data

            str1 = 'Tasa'
            tasa = ''
            for i in this_data.columns:
                ratio1 = fuzz.partial_ratio(str1.lower(), str(i).lower())
                if ratio1 >= 75:
                    print(i)
                    tasa = i
            print(tasa)

            if tasa != '':
                tasa_col = this_data[tasa]
                tasa_data = pd.DataFrame(tasa_col, columns=[tasa_col])
                tasa_data



            df2 = this_data
            if disc == '' and imposto == '' and tasa == '':
                df2 = pd.concat([codingo_data, dataf, Cantidad_data, price_data], axis=1, join='inner')
                df2['Discount'] = 0
                df2['IMPUESTO'] = 0
                df2['Tasa'] = 0
                print('---HI------')

            elif disc != '' and imposto == '' and tasa == '':
                disc_col = this_data[disc]
                disc_data = pd.DataFrame(disc_col, columns=[disc])
                df2 = pd.concat([codingo_data, dataf, Cantidad_data, price_data, disc_data], axis=1, join='inner')
                df2['IMPUESTO'] = 0
                df2['Tasa'] = 0
                print('---HI------')

            elif disc != '' and imposto != '' and tasa == '':
                disc_col = this_data[disc]
                disc_data = pd.DataFrame(disc_col, columns=[disc])
                df2 = pd.concat([codingo_data, dataf, Cantidad_data, price_data, disc_data, imposto_data], axis=1, join='inner')
                df2['Tasa'] = 0
                print('---HI------')

            elif disc != '' and imposto == '' and tasa != '':
                disc_col = this_data[disc]
                disc_data = pd.DataFrame(disc_col, columns=[disc])
                df2 = pd.concat([codingo_data, dataf, Cantidad_data, price_data, disc_data, tasa_data], axis=1, join='inner')
                df2['IMPUESTO'] = 0
                print('---HI------')

            elif disc == '' and imposto != '' and tasa != '':
                df2 = pd.concat([codingo_data, dataf, Cantidad_data, price_data, imposto_data, tasa_data], axis=1, join='inner')
                df2['Discount'] = 0
                print('---HI------')

            # data['Vat Number']= data['Vat Number'].str.replace(',', '')
            df2.columns.values[0] = "Codigo"
            df2.columns.values[1] = "Description"
            df2.columns.values[2] = "Precio"
            df2.columns.values[3] = "Cantidad"
            df2.columns.values[4] = "Discount"
            print('df2 this',df2)       
            if df2 is None:
                print('df2 is null')
    return df2

#read_image('abc.png')